In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
landmarks = pd.read_csv('../../Data/train.csv')
landmarks.landmark_id = landmarks.landmark_id.astype(str)
landmarks

id landmark_id
0        17660ef415d37059           1
1        92b6290d571448f6           1
2        cd41bf948edc0340           1
3        fb09f1e98c6d2f70           1
4        25c9dfc7ea69838d           7
...                   ...         ...
1580465  72c3b1c367e3d559      203092
1580466  7a6a2d9ea92684a6      203092
1580467  9401fad4c497e1f9      203092
1580468  aacc960c9a228b5f      203092
1580469  d9e338c530dca106      203092

[1580470 rows x 2 columns]

In [3]:
def get_file(id):
    root = 'C:/Users/samma/Desktop/Data science projects/landmark-recognition-2021/Data/train/'
    subfolder ='/'.join(x for x in id[0:3])
    file = '/'+id+'.jpg'
    return root+subfolder+file

In [4]:
get_file(landmarks.id[0])

'C:/Users/samma/Desktop/Data science projects/landmark-recognition-2021/Data/train/1/7/6/17660ef415d37059.jpg'

In [5]:
landmarks['file'] = landmarks.id.apply(get_file)
landmarks

id landmark_id  \
0        17660ef415d37059           1   
1        92b6290d571448f6           1   
2        cd41bf948edc0340           1   
3        fb09f1e98c6d2f70           1   
4        25c9dfc7ea69838d           7   
...                   ...         ...   
1580465  72c3b1c367e3d559      203092   
1580466  7a6a2d9ea92684a6      203092   
1580467  9401fad4c497e1f9      203092   
1580468  aacc960c9a228b5f      203092   
1580469  d9e338c530dca106      203092   

                                                      file  
0        C:/Users/samma/Desktop/Data science projects/l...  
1        C:/Users/samma/Desktop/Data science projects/l...  
2        C:/Users/samma/Desktop/Data science projects/l...  
3        C:/Users/samma/Desktop/Data science projects/l...  
4        C:/Users/samma/Desktop/Data science projects/l...  
...                                                    ...  
1580465  C:/Users/samma/Desktop/Data science projects/l...  
1580466  C:/Users/samma/Desktop/Data science projects/l...  
1580467  C:/Users/samma/Desktop/Data science projects/l...  
1580468  C:/Users/samma/Desktop/Data science projects/l...  
1580469  C:/Users/samma/Desktop/Data science projects/l...  

[1580470 rows x 3 columns]

In [6]:
train, valid = train_test_split(landmarks,test_size=0.2, random_state=77)
valid=valid[~valid.landmark_id.isin(list(set(valid.landmark_id.values)-set(train.landmark_id.values)))]
print(len(train.landmark_id.value_counts()))
print(len(train))
print(len(valid.landmark_id.value_counts()))
print(len(valid))

81076
1264376
65713
315560


In [7]:
datagen = ImageDataGenerator(rescale=1/255)

train_generator = datagen.flow_from_dataframe(dataframe=train, x_col='file', y_col='landmark_id', class_mode='categorical',
                                             target_size = (300,300), batch_size=32)
validation_generator = datagen.flow_from_dataframe(dataframe=valid, x_col='file',  y_col='landmark_id', class_mode='categorical',
                                                  target_size=(300,300), batch_size=32)

Found 1264376 validated image filenames belonging to 81076 classes.
Found 315560 validated image filenames belonging to 65713 classes.


In [8]:
def GAP(predictions, confidence, actuals):
    total = 0
    for i in range(0,len(predictions)):
        if predictions[i] == actuals[i]:
            total += confidence[i]
    GAP = total/len(predictions)
    return GAP

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(81, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='CategoricalCrossentropy', optimizer = 'Adam', metrics = ['Precision'])

In [ ]:
history = model.fit(small_train_generator, epochs = 100, steps_per_epoch=10, validation_data=small_validation_generator, validation_steps=10)

In [ ]:
history.history

In [ ]:
predictions = model.predict(small_validation_generator)

In [ ]:
pred=[]
percent = []
for prediction in predictions:
    pred.append(np.unique(small_train.landmark_id)[np.argmax(prediction)])
    percent.append(max(prediction))

In [ ]:
pred

In [ ]:
percent

In [ ]:
GAP(pred,percent,small_valid.landmark_id.values)